## Import Libraries

In [13]:
import matplotlib.pyplot as plt
import numpy as np
import geopandas
import rasterio
import GUI_test
import os
import imageio

## Upload Data

In [14]:
def open_geoTiff(filepath):
    file_src = rasterio.open(filepath)
    meta_data = file_src.profile
    bands = range(1, meta_data['count']+1)
    image = file_src.read(bands)
    image = np.transpose(image, (1,2,0))
    file_src.close()
    return image, meta_data 
    
if not 'poles' in locals(): 
    poles = geopandas.read_file("poles.gpkg")
    
plt.close('all')
dataset_opt = []
dataset_SAR = []
dataset_SAR_processed = []


def is_window_ok(row, col, meta_data, radius):
    if (col > radius_window and col < meta_data['width'] - radius_window and
    row > radius_window and row < meta_data['height'] - radius_window):
        return True
    else:
        return False
    
image_files = ["optical.tif","SAR_clipped_1.tif","SAR_preprocessed_1.tif"]   
dataset=[]

## Extract windows and create datasets

In [15]:
for image_filename in image_files:
    print("Process " + image_filename)
    image, meta_data = open_geoTiff(image_filename)
    
    for index, _poles in poles.iterrows():
    
        x = _poles['geometry'].x
        y = _poles['geometry'].y
        
        col = int((x - meta_data['transform'][2])/meta_data['transform'][0]) 
        row = int((y - meta_data['transform'][5])/meta_data['transform'][4]) 
    
        radius_window = int(50/meta_data['transform'][0])
    
        if is_window_ok(row, col, meta_data, radius_window):
            
            # Extarct window from optical
            window = image[row-radius_window:row+radius_window, col-radius_window:col+radius_window,:]
            dataset.append(window)      
    
        else:
            print("WARNING: point too close to the boundary")

n = int(len(dataset) / 3)
dataset_opt = dataset[0:n] 
dataset_SAR = dataset[n:2*n] 
dataset_SAR_processed = dataset[2*n::] 
 
dataset_opt = np.asarray(dataset_opt)
dataset_SAR = np.asarray(dataset_SAR)
dataset_SAR_processed = np.asarray(dataset_SAR_processed)


# Load your NumPy arrays
dataset_opt = np.asarray(dataset_opt)
dataset_SAR = np.asarray(dataset_SAR)
dataset_SAR_processed = np.asarray(dataset_SAR_processed)

Process optical.tif
Process SAR_clipped_1.tif
Process SAR_preprocessed_1.tif


## Save Dataset as Tiff

In [16]:
#Specify the directory where you want to save the TIFF images
output_directory = "output_tiff_images/"

#Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

#Function to save optical windows as TIFF
def save_image_as_tiff(image, filename):
     imageio.imwrite(os.path.join(output_directory, filename), image, format="TIFF")

#Iterate through the datasets and save each image as TIFF
for i, image_opt in enumerate(dataset_opt):
     save_image_as_tiff(image_opt, f"image_opt_{i}.tiff")

## Show visualization on GUI

In [19]:
GUI_test.visualize_dataset(dataset_opt, dataset_SAR, dataset_SAR_processed)

<img src="GUI.png" alt="GUI Image" width="600">